In [ ]:
!pip3 install datasets
!pip3 install transformers
!pip3 install sentencepiece
!pip3 install protobuf
!pip3 install torch
!pip3 install hindi-stemmer
!pip3 install indic-nlp-library
!pip3 install snowballstemmer

In [3]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode
import torch
from indicnlp.tokenize import indic_tokenize
from indicnlp import common
from indicnlp import loader
from indicnlp.morph import unsupervised_morph
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from snowballstemmer import stemmer
import pandas as pd
import re

/Users/mitanshk/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mitanshk/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
lang='hi'
raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

In [5]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [39]:
train_data = raw_datasets['train']
val_data = raw_datasets['validation']
test_data = raw_datasets['test']

In [41]:
print(train_data[0])

{'tokens': ['सेक्टर', '55/56', 'के', 'एसएचओ', 'अरविंद', 'कुमार', 'ने', 'बताया', 'कि', 'इस', 'मामले', 'में', 'आईपीसी', 'की', 'धारा', '376', '-', 'डी', '(', 'गैंगरेप', ')', 'के', 'तहत', 'मामला', 'दर्ज', 'कर', 'लिया', 'गया', 'है', '।'], 'ner_tags': [0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [7]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [8]:
label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


## Performing tokenization on our datset

In [9]:
config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
padding = "max_length"

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # Ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
print(raw_datasets["train"])

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 985787
})


In [42]:
train_dataset = raw_datasets["train"]
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

In [43]:
test_dataset = raw_datasets["test"]
test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

In [44]:
print(train_dataset)

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 985787
})


In [ ]:
df = pd.DataFrame(train_dataset)
df.to_csv('cleaned_data.csv', index=False)

df = pd.DataFrame(test_data)
df.to_csv('test_data.csv', index=False)

In [13]:
print("Train dataset after tokenization")
print(train_dataset)

Train dataset after tokenization
Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 985787
})


In [14]:
print(train_dataset[0]['ner_tags'])

[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Performing Normalization on our dataset

In [15]:
normalizer_factory = IndicNormalizerFactory()
normalizer = normalizer_factory.get_normalizer("hi",remove_nuktas=False)

for entry in train_dataset:
    tokens = entry['tokens']
    normalized_tokens = [normalizer.normalize(token) for token in tokens]
    entry['tokens'] = normalized_tokens

In [21]:
for entry in test_dataset:
    tokens = entry['tokens']
    normalized_tokens = [normalizer.normalize(token) for token in tokens]
    entry['tokens'] = normalized_tokens

## Performing Morph Analyzer on tokens to see their root form

In [16]:
INDIC_NLP_RESOURCES = r"./indic_nlp_resources-master/"
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()

analyzer = unsupervised_morph.UnsupervisedMorphAnalyzer('hi')

In [17]:
sample_dataset = train_dataset.select(range(10))

def morph_hindi_tokens(tokens_list):
    stemmed_tokens = []
    for tokens in tokens_list:
        stemmed_tokens_per_list = []
        for token in tokens:
            stem = analyzer.morph_analyze(token)
            stemmed_tokens_per_list.append(" ".join([s[0] for s in stem]))
        stemmed_tokens.append(stemmed_tokens_per_list)
    return stemmed_tokens

morphed_dataset = sample_dataset.map(
    lambda example: {
        'tokens': morph_hindi_tokens(example['tokens']),
        'ner_tags': example['ner_tags'],
        'input_ids': example['input_ids'],
        'token_type_ids': example['token_type_ids'],
        'attention_mask': example['attention_mask'],
        'labels': example['labels'],
    },
    batched=True
)

print(morphed_dataset)

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10
})


In [18]:
print(sample_dataset[0])
print(morphed_dataset[0])

{'tokens': ['सेक्टर', '55/56', 'के', 'एसएचओ', 'अरविंद', 'कुमार', 'ने', 'बताया', 'कि', 'इस', 'मामले', 'में', 'आईपीसी', 'की', 'धारा', '376', '-', 'डी', '(', 'गैंगरेप', ')', 'के', 'तहत', 'मामला', 'दर्ज', 'कर', 'लिया', 'गया', 'है', '।'], 'ner_tags': [0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_ids': [2, 30265, 8430, 4415, 429, 13043, 1883, 90621, 13734, 109832, 22901, 236, 78701, 1134, 1883, 70, 2092, 29073, 2092, 941, 89842, 1883, 14855, 69922, 13, 4086, 20, 2344, 8213, 1551, 28, 1883, 1694, 2092, 29073, 1902, 1325, 68, 55450, 2344, 1134, 4384, 7, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## Performing stemming on our dataset

In [14]:
lang_stemmer = stemmer('hindi')

def stem_hindi_tokens(tokens_list):
    stemmed_tokens = []
    for tokens in tokens_list:
        stemmed_tokens_per_list = []
        for token in tokens:
            stemmed_token = lang_stemmer.stemWord(token)
            stemmed_tokens_per_list.append(stemmed_token)
        stemmed_tokens.append(stemmed_tokens_per_list)
    return stemmed_tokens

df = pd.DataFrame({
    'tokens': train_dataset['tokens'],
    'ner_tags': train_dataset['ner_tags'],
    'input_ids': train_dataset['input_ids'],
    'token_type_ids': train_dataset['token_type_ids'],
    'attention_mask': train_dataset['attention_mask'],
    'labels': train_dataset['labels'],
})

df['tokens'] = df['tokens'].apply(stem_hindi_tokens)

stemmed_dataset = {
    'tokens': df['tokens'],
    'ner_tags': df['ner_tags'],
    'input_ids': df['input_ids'],
    'token_type_ids': df['token_type_ids'],
    'attention_mask': df['attention_mask'],
    'labels': df['labels'],
}

: 

: 

In [31]:
lang_stemmer = stemmer('hindi')

def stem_hindi_tokens(tokens_list):
    stemmed_tokens = []
    for tokens in tokens_list:
        stemmed_tokens_per_list = []
        for token in tokens:
            stemmed_token = lang_stemmer.stemWord(token)
            stemmed_tokens_per_list.append(stemmed_token)
        stemmed_tokens.append(stemmed_tokens_per_list)
    return stemmed_tokens

df = pd.DataFrame({
    'tokens': test_dataset['tokens'],
    'ner_tags': test_dataset['ner_tags'],
    'input_ids': test_dataset['input_ids'],
    'token_type_ids': test_dataset['token_type_ids'],
    'attention_mask': test_dataset['attention_mask'],
    'labels': test_dataset['labels'],
})

df['tokens'] = df['tokens'].apply(stem_hindi_tokens)

stemmed_dataset = {
    'tokens': df['tokens'],
    'ner_tags': df['ner_tags'],
    'input_ids': df['input_ids'],
    'token_type_ids': df['token_type_ids'],
    'attention_mask': df['attention_mask'],
    'labels': df['labels'],
}

In [38]:
print(stemmed_dataset)
print(test_data[10])

{'tokens': 0      [[ग, ु, ज, र, ा, त], [ह, ा, ई, क, ो, र, ्, ट],...
1      [[श, ा, ह, र, ु, ख], [ख, ा, न], [इ, न], [द, ि,...
2      [[प, र, ि, ण, ी, त, ि], [न, े], [क, ह, ा], [क,...
3      [[स, ो, श, ल], [म, ी, ड, ि, य, ा], [प, र], [स,...
4      [[प, र, ी, क, ्, ष, ण], [क, े], [द, ौ, र, ा, न...
                             ...                        
862    [[2, 0, 1, 0], [क, े], [च, ु, न, ा, व, ो, ं], ...
863    [[ह, ा, ई], [ब, ्, ल, ड], [प, ्, र, े, श, र], ...
864    [[उ, न, ्, ह, ो, ं, न, े], [न, न, क, ा, न, ा],...
865    [[B, J, P], [म, म, त, ा], [स, र, क, ा, र], [क,...
866    [[इ, न, म, े, ं], [स, े], [ए, क], [ग, ु, ट], [...
Name: tokens, Length: 867, dtype: object, 'ner_tags': 0      [3, 4, 0, 0, 0, 3, 0, 1, 2, 0, 0, 0, 0, 0, 0, ...
1             [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2          [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3      [0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 1, 2, ...
4             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                       

In [1]:
print(train_dataset[10])
print(stemmed_dataset[10])  

NameError: name 'train_dataset' is not defined

## Removal of noisy words from input data

In [25]:
special_characters = ['nn', 'n', '।', '/', '`', '+', '\\', '"', '?', '▁(', '$', '@', '[', '_', "'", '!', ',', ':', '^', '|', ']', '=', '%', '&', '.', ')', '(', '#', '*', '', ';', '-', '}', '|', '"']

pattern = '|'.join(re.escape(char) for char in special_characters)
pattern = f"[{pattern}]"

In [26]:
def clean_hindi_tokenized_array(tokenized_array):
    cleaned_tokens = []
    for token in tokenized_array:
        token = re.sub(r'<[^>]+>', '', token) 
        token = re.sub(pattern, '', token) 
        token = ' '.join(token.split())
        cleaned_tokens.append(token)

    return cleaned_tokens

In [29]:
cleaned_data_test = []

for line in test_dataset:
    cleaned_line = clean_hindi_tokenized_array(line['tokens'])
    line['tokens'] = cleaned_line
    cleaned_data_test.append(line)

In [1]:
cleaned_data = []

for line in train_dataset:
    cleaned_line = clean_hindi_tokenized_array(line['tokens'])
    line['tokens'] = cleaned_line
    cleaned_data.append(line)

NameError: name 'train_dataset' is not defined

In [14]:
print(train_dataset[0])
print(cleaned_data[0])

{'tokens': ['सेक्टर', '55/56', 'के', 'एसएचओ', 'अरविंद', 'कुमार', 'ने', 'बताया', 'कि', 'इस', 'मामले', 'में', 'आईपीसी', 'की', 'धारा', '376', '-', 'डी', '(', 'गैंगरेप', ')', 'के', 'तहत', 'मामला', 'दर्ज', 'कर', 'लिया', 'गया', 'है', '।'], 'ner_tags': [0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_ids': [2, 30265, 8430, 4415, 429, 13043, 1883, 90621, 13734, 109832, 22901, 236, 78701, 1134, 1883, 70, 2092, 29073, 2092, 941, 89842, 1883, 14855, 69922, 13, 4086, 20, 2344, 8213, 1551, 28, 1883, 1694, 2092, 29073, 1902, 1325, 68, 55450, 2344, 1134, 4384, 7, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
# save the clean data and test data as csv files
df = pd.DataFrame(cleaned_data)
df.to_csv('cleaned_data.csv', index=False)

df = pd.DataFrame(test_data)
df.to_csv('test_data.csv', index=False)

df = pd.DataFrame(val_data)
df.to_csv('val_data.csv', index=False)

: 

In [30]:
df = pd.DataFrame(cleaned_data_test)
df.to_csv('test_data.csv', index=False)